In [27]:
from bs4 import BeautifulSoup
with open('data/linkedin_connect.html') as f: 
    soup=BeautifulSoup(f, 'html.parser')
reactions=soup.find_all('li', class_='artdeco-list__item')
reaction_list=[]
for each_reaction in reactions: 
    reaction={'link': each_reaction.find('a')['href'], 
              'name': each_reaction.find('div', class_='artdeco-entity-lockup__title').span.text.strip(), 
              'connection': each_reaction.find('span', class_='artdeco-entity-lockup__degree').text.strip().replace('·\xa0', ''), 
              'description': each_reaction.find('div', class_='artdeco-entity-lockup__caption').text.strip(), 
              'article_link': 'https://www.linkedin.com/feed/update/urn:li:activity:6804666680736833536/'}
    if '1st' in reaction['connection']: 
        reaction['connected']=True
    else: 
        reaction['connected']=False
    reaction_list.append(reaction)

In [29]:
import pandas as pd
df=pd.DataFrame(reaction_list)

In [6]:
articles_list={'https://www.linkedin.com/feed/update/urn:li:activity:6804666680736833536/': 'sentiment analysis'}

In [7]:
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import time
# browser = init_browser()
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472
[WDM] - Driver [/Users/kevinlee/.wdm/drivers/chromedriver/mac64/91.0.4472.19/chromedriver] found in cache


In [8]:
from dotenv import load_dotenv
import os
load_dotenv()
username=os.environ['username']
password=os.environ['password']

In [97]:
from nameparser import HumanName
import random
MAX_WAIT=10
MIN_WAIT=5
def generate_invite(name, article): 
    return f'Hi {name.capitalize()} - I wanted to thank you for taking the time to read and like my article on {article.lower()}. I enjoy learning about what other data-focused people are interested in and working on. I would love to make a connection.'
def get_alias(full_name):
    name=HumanName(full_name)
    if name.nickname: 
        return name.nickname.capitalize()
    else: 
        return name['first'].capitalize()
def get_article_summary(link): 
    return articles_list[link]    
def browser_init(): 
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    return browser    
def login(browser, username, password): 
    url='https://www.linkedin.com/'
    browser.visit(url)
    time.sleep(1)

    browser.find_by_id('session_key').type(username)
    time.sleep(1)
    browser.find_by_id('session_password').type(password)
    time.sleep(1)
    browser.find_by_css('button[class="sign-in-form__submit-button"]').click()
    time.sleep(1)
    return True
# browser.find_by_id('session_key').type('@gmail.com', slowly=True)#click()
# browser.type('kevin.c.lee26@gmail.com', slowly=True)

In [103]:
current_browser=browser_init()
login(current_browser, username, password)
time.sleep(random.randint(MIN_WAIT, MAX_WAIT))

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472
[WDM] - Driver [/Users/kevinlee/.wdm/drivers/chromedriver/mac64/91.0.4472.19/chromedriver] found in cache


In [109]:
def request_connection(browser, record): 
    alias=get_alias(record['name'])
    # check if S/S shows connected
    # if not connected
    if not record['connected']: 
        current_browser.visit(record['link'])
        time.sleep(random.randint(MIN_WAIT, MAX_WAIT))
        # profile is the container div
        profile=current_browser.find_by_css('.pvs-profile-actions')
        # check if actually not connected
        #
        #
        # check if pending
        # check if connect button exists
        # connecxt is a button (connect/pending)
        connect=profile.find_by_css('button[data-control-name="connect"]')
        # if element is not found, find will return an empty list
        # if connect button exits, then 2nd connection: 
        if len(connect) > 0: 
            # check if pending
            if connect.text=='Pending': 
                print(f'The connect request to {alias} is pending')
                record['pending']=True
            # check if connect
            elif connect.text=='Connect': 
                print(f'Sending request')
                connect.click()
                time.sleep(random.randint(MIN_WAIT, MAX_WAIT))
                invite_dialog=current_browser.find_by_css('.send-invite')
                add_note=invite_dialog.find_by_text('Add a note')
                add_note.click()
                time.sleep(random.randint(MIN_WAIT, MAX_WAIT))
                custom_message=invite_dialog.find_by_id('custom-message')
                custom_message.type(generate_invite(alias, get_article_summary(record['article_link'])))
                time.sleep(random.randint(MIN_WAIT, MAX_WAIT))
                send=invite_dialog.find_by_text('Send')
                send.click()
                record['pending']=True
            # unlikely to be neither connect or pending but just in case
            else: 
                pass
        # else this is a 3rd connection
        else: 
            print(f'No connect button')
            # more is a button
            more=profile.find_by_css('.artdeco-dropdown__trigger')
            more.click()
            time.sleep(random.randint(MIN_WAIT, MAX_WAIT))
            # connect is same layer as more
            connect=profile.find_by_css('div[data-control-name="connect"]')
            if len(connect)>0: 
                if 'connect' in connect.text.lower(): 
                    connect.click()
                    # might need to wait longer here
                    time.sleep(random.randint(MIN_WAIT, MAX_WAIT))
                    invite_dialog=current_browser.find_by_css('.send-invite')
                    # if a dialog pops up then it's not pending
                    # if verification pops up then it's no good
                    verification=invite_dialog.find_by_name('email')
                    if not len(verification)>0: 
                        print(f'Sending invite')
                        add_note=invite_dialog.find_by_text('Add a note')
                        add_note.click()
                        time.sleep(random.randint(MIN_WAIT, MAX_WAIT))
                        custom_message=invite_dialog.find_by_id('custom-message')
                        custom_message.type(generate_invite(alias, get_article_summary(record['article_link'])))
                        time.sleep(random.randint(MIN_WAIT, MAX_WAIT))
                        send=invite_dialog.find_by_text('Send')
                        send.click()
                        record['pending']=True
                    else: 
                        print(f'This member wants verification for the invite')
                else: 
                    # pending, as in found in text
#                     'pending' in connect.text.lower(): 
                    record['pending']=True
                    print(f'The connect request to {alias} is pending')
            else: 
                print(f'No more -> connect button')
    # already connected
    else: 
        record['pending']=False
        print(f"You're connected with {alias}")
    return True

In [101]:
generate_invite(get_alias(sample['name']), get_article_summary(sample['article_link']))




'Hi Nicola - I wanted to thank you for taking the time to read and like my article on sentiment analysis. I enjoy learning about what other data-focused people are interested in and working on. I would love to make a connection.'

In [1]:
sample=df.loc[54].to_dict()

NameError: name 'df' is not defined

In [110]:
request_connection(current_browser, sample)

No connect button
This member wants verification for the invite


True

In [112]:
for idx, each_record in df.iterrows(): 
    if (idx>73): # & (idx<20):
        print(f'-====={idx}=====-')
        request_connection(current_browser, each_record.to_dict())

-=====74=====-
No connect button
Sending invite
-=====75=====-
No connect button
Sending invite
-=====76=====-
No connect button
Sending invite
-=====77=====-
No connect button
Sending invite
-=====78=====-
Sending request
-=====79=====-
No connect button
Sending invite
-=====80=====-
No connect button
Sending invite
-=====81=====-
No connect button
Sending invite
-=====82=====-
No connect button
Sending invite
-=====83=====-
No connect button
Sending invite
-=====84=====-
No connect button
Sending invite
-=====85=====-
No connect button
Sending invite
-=====86=====-
No connect button
Sending invite
-=====87=====-
No connect button
Sending invite
-=====88=====-
No connect button
Sending invite
-=====89=====-
No connect button
Sending invite
-=====90=====-
No connect button
Sending invite
-=====91=====-
No connect button
Sending invite
-=====92=====-
No connect button
Sending invite
-=====93=====-
No connect button
Sending invite
-=====94=====-
Sending request
-=====95=====-
No connect 

KeyboardInterrupt: 

In [93]:
len(current_browser.find_by_css('.artdeco-dropdown__trigger'))

3

In [172]:
df.to_csv('data/list.csv', index=False)

In [2]:
# df.head()